In [1]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv("train.csv")
y = data['SalePrice']
X = data.drop('SalePrice', axis=1)
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
X.drop('Id', axis=1)
numeric_cols = list(X.select_dtypes(include=np.number).columns)
X_num = X[numeric_cols]
X_num.columns = X[numeric_cols].columns
X_num.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


## First model will use only numeric values and missing values will be imputed with SimpleImputer

In [4]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

def get_train_test_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_test.columns = X_test.columns
    
    X_train, X_test = imputed_X_train, imputed_X_test
    
    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
    y_test = torch.tensor(y_test.values, dtype=torch.float32).squeeze()
    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)
X_num = pd.DataFrame(my_imputer.transform(X_num))

In [6]:
class ModelV1(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=1),
            # nn.Linear(in_features=10, out_features=10),
            # nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [7]:
model1 = ModelV1(in_features=37)

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model1.parameters(), lr=0.0001)

In [8]:
model1.eval()
y_pred = model1(X_train).squeeze()
y_pred

tensor([ -474.0264,  -650.4340,  -734.6797,  ...,  -835.0174,  -503.3602,
        -1073.6823], grad_fn=<SqueezeBackward0>)

In [9]:
def train_test(model:torch.nn.Module, loss_fn:torch.nn.Module, optimizer:torch.optim.Optimizer, epochs:int, X_train, X_test, y_train, y_test):
    
    for epoch in range(epochs):
        model.train()
        y_preds = model(X_train).squeeze()
        loss = loss_fn(y_preds, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with (torch.inference_mode()):
            test_pred = model(X_test).squeeze()
            test_loss = loss_fn(test_pred, y_test)
        if epoch % 50 == 0:
            print("----------------------------")
            print(f"Epoch {epoch}:")
            print(f"Train loss: {loss} | Test loss: {test_loss}")

In [10]:
train_test(model=model1, loss_fn=loss_fn, optimizer=optimizer, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 181759.0625 | Test loss: 167535.671875
----------------------------
Epoch 50:
Train loss: 61975.453125 | Test loss: 49551.8828125
----------------------------
Epoch 100:
Train loss: 57334.12890625 | Test loss: 46591.12109375
----------------------------
Epoch 150:
Train loss: 53421.984375 | Test loss: 44410.29296875
----------------------------
Epoch 200:
Train loss: 50357.42578125 | Test loss: 43212.046875
----------------------------
Epoch 250:
Train loss: 48113.765625 | Test loss: 42274.8359375
----------------------------
Epoch 300:
Train loss: 46456.4921875 | Test loss: 41690.92578125
----------------------------
Epoch 350:
Train loss: 45199.046875 | Test loss: 41291.29296875
----------------------------
Epoch 400:
Train loss: 44108.54296875 | Test loss: 40899.99609375
----------------------------
Epoch 450:
Train loss: 43169.12890625 | Test loss: 40496.234375
----------------------------
Epoch 500:
Train loss: 42391.23828125 | Tes

## This results are not great, let's figure out which 5 data columns have the most corelation with SalePrice and use only them

In [11]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [12]:
discrete_features = X_num.dtypes == int

mi_scores = make_mi_scores(X_num, y, discrete_features=discrete_features)
mi_scores[:7]

4     0.563859
16    0.481432
12    0.368610
27    0.359944
6     0.357934
26    0.354998
13    0.309227
Name: MI Scores, dtype: float64

## Let's now use only this 7 features in a Model and see if there is any difference in performance

In [13]:
model2 = ModelV1(in_features=7)
lista = list(mi_scores[:7].index)
for i in lista:
    i -= 1
X_num_2 = X_num[lista]
X_train, X_test, y_train, y_test = get_train_test_data(X_num_2, y, test_size=0.2)
optimizer_2 = torch.optim.SGD(params=model2.parameters(), lr=0.001)

In [14]:
train_test(model=model2, loss_fn=loss_fn, optimizer=optimizer_2, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 183140.328125 | Test loss: 165394.46875
----------------------------
Epoch 50:
Train loss: 38650.90625 | Test loss: 37497.1171875
----------------------------
Epoch 100:
Train loss: 36891.08984375 | Test loss: 35779.34375
----------------------------
Epoch 150:
Train loss: 35508.72265625 | Test loss: 34315.71875
----------------------------
Epoch 200:
Train loss: 34434.8828125 | Test loss: 33131.171875
----------------------------
Epoch 250:
Train loss: 33569.90625 | Test loss: 32203.740234375
----------------------------
Epoch 300:
Train loss: 32845.20703125 | Test loss: 31391.423828125
----------------------------
Epoch 350:
Train loss: 32299.798828125 | Test loss: 30840.66796875
----------------------------
Epoch 400:
Train loss: 31864.28125 | Test loss: 30444.599609375
----------------------------
Epoch 450:
Train loss: 31513.61328125 | Test loss: 30128.5078125
----------------------------
Epoch 500:
Train loss: 31223.94921875 | Tes

## Now let's try to normalize data and see whether it improves our result or not

In [15]:
X_num_3 = X_num_2
y_normalized = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(y, dtype=torch.float32).unsqueeze(dim=1)))
X_train, X_test, y_train, y_test = get_train_test_data(X_num_3, y_normalized, test_size=0.2)
X_train, X_test = torch.nn.functional.normalize(X_train), torch.nn.functional.normalize(X_test)

In [16]:
model3 = ModelV1(in_features=7)
optimizer_3 = torch.optim.SGD(params=model3.parameters(), lr=0.001)

In [17]:
train_test(model=model3, loss_fn=loss_fn, optimizer=optimizer_3, epochs=2001, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 0.485415518283844 | Test loss: 0.4867783188819885
----------------------------
Epoch 50:
Train loss: 0.38735586404800415 | Test loss: 0.3886602222919464
----------------------------
Epoch 100:
Train loss: 0.28929615020751953 | Test loss: 0.2905420660972595
----------------------------
Epoch 150:
Train loss: 0.1912364661693573 | Test loss: 0.19242393970489502
----------------------------
Epoch 200:
Train loss: 0.09317672997713089 | Test loss: 0.09430575370788574
----------------------------
Epoch 250:
Train loss: 0.031522758305072784 | Test loss: 0.030014952644705772
----------------------------
Epoch 300:
Train loss: 0.02891691029071808 | Test loss: 0.025968892499804497
----------------------------
Epoch 350:
Train loss: 0.02837458997964859 | Test loss: 0.025271162390708923
----------------------------
Epoch 400:
Train loss: 0.027866709977388382 | Test loss: 0.02476171962916851
----------------------------
Epoch 450:
Train loss: 0.02736

## Let's try now with more complicated model

In [18]:
X_train, X_test, y_train, y_test = get_train_test_data(X_num, y, test_size=0.2)

In [19]:
class ModelV2(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=10),
            nn.Linear(in_features=10, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, x):
        return self.layer(x)

In [20]:
model4 = ModelV2(37)
loss_fn = nn.L1Loss()
optimizer_4 = torch.optim.Adam(model4.parameters(), lr=0.01, weight_decay=1e-5)

In [21]:
train_test(model4, loss_fn=loss_fn, optimizer=optimizer_4, epochs=2000, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 182205.359375 | Test loss: 177054.1875
----------------------------
Epoch 50:
Train loss: 47997.8359375 | Test loss: 48034.96875
----------------------------
Epoch 100:
Train loss: 40956.8203125 | Test loss: 40956.19921875
----------------------------
Epoch 150:
Train loss: 33094.8203125 | Test loss: 33707.1796875
----------------------------
Epoch 200:
Train loss: 28126.640625 | Test loss: 30191.05859375
----------------------------
Epoch 250:
Train loss: 27174.7421875 | Test loss: 29409.228515625
----------------------------
Epoch 300:
Train loss: 26952.630859375 | Test loss: 29026.658203125
----------------------------
Epoch 350:
Train loss: 26822.541015625 | Test loss: 28795.314453125
----------------------------
Epoch 400:
Train loss: 26715.044921875 | Test loss: 28538.818359375
----------------------------
Epoch 450:
Train loss: 26634.185546875 | Test loss: 28318.1328125
----------------------------
Epoch 500:
Train loss: 26581.32

## Let's now try using categorical variables

In [34]:
from sklearn.preprocessing import OrdinalEncoder

label_X = X.copy()

s = (X.dtypes == 'object')
object_cols = list(s[s].index)

ordinal_encoder = OrdinalEncoder()
label_X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])
label_X = label_X.drop('Id', axis=1)
label_X = pd.DataFrame(my_imputer.fit_transform(label_X))
label_X.head()

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,60.0,3.0,65.0,8450.0,1.0,0.450549,3.0,3.0,0.0,4.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2008.0,8.0,4.0
1,20.0,3.0,80.0,9600.0,1.0,0.450549,3.0,3.0,0.0,2.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,5.0,2007.0,8.0,4.0
2,60.0,3.0,68.0,11250.0,1.0,0.450549,0.0,3.0,0.0,4.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,9.0,2008.0,8.0,4.0
3,70.0,3.0,60.0,9550.0,1.0,0.450549,0.0,3.0,0.0,0.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,2.0,2006.0,8.0,0.0
4,60.0,3.0,84.0,14260.0,1.0,0.450549,0.0,3.0,0.0,2.0,...,0.0,0.0,1.142857,1.427046,1.907407,0.0,12.0,2008.0,8.0,4.0


In [38]:
mi_scores_all = make_mi_scores(label_X, y, discrete_features=label_X.columns)
mi_scores_all[:10]

16    0.571125
11    0.527699
61    0.494664
45    0.430655
18    0.409119
37    0.398195
3     0.393821
60    0.369533
58    0.343536
29    0.334620
Name: MI Scores, dtype: float64

In [49]:
useful_data = list(mi_scores_all[mi_scores_all.gt(0.3)].index)
label_X = label_X[useful_data]

In [51]:
X_train, X_test, y_train, y_test = get_train_test_data(label_X, y, test_size=0.2)

In [63]:
model5 = ModelV2(len(useful_data))
optimizer_5 = torch.optim.Adam(params=model5.parameters(), lr=0.02, weight_decay=1e-4)

In [64]:
train_test(model5, loss_fn=loss_fn, optimizer=optimizer_5, epochs=2001, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

----------------------------
Epoch 0:
Train loss: 182954.359375 | Test loss: 174798.0625
----------------------------
Epoch 50:
Train loss: 52202.23046875 | Test loss: 53017.59765625
----------------------------
Epoch 100:
Train loss: 35237.18359375 | Test loss: 34217.0625
----------------------------
Epoch 150:
Train loss: 28863.908203125 | Test loss: 29209.896484375
----------------------------
Epoch 200:
Train loss: 28679.1484375 | Test loss: 29233.00390625
----------------------------
Epoch 250:
Train loss: 28601.75390625 | Test loss: 29230.91015625
----------------------------
Epoch 300:
Train loss: 28551.373046875 | Test loss: 29231.30859375
----------------------------
Epoch 350:
Train loss: 28492.83203125 | Test loss: 29168.60546875
----------------------------
Epoch 400:
Train loss: 28452.486328125 | Test loss: 29091.44921875
----------------------------
Epoch 450:
Train loss: 28387.71484375 | Test loss: 29106.2421875
----------------------------
Epoch 500:
Train loss: 28365.2